In [88]:
%load_ext autoreload
%autoreload 2

import csv
from pathlib import Path
import re
import torch
import torch.nn as nn
import torch_fidelity
import torchvision
from torchvision.models.feature_extraction import get_graph_node_names
from torchvision.models.feature_extraction import create_feature_extractor
from torchvision.models.detection.mask_rcnn import MaskRCNN
from torchvision.models.detection.backbone_utils import LastLevelMaxPool
from torchvision.ops.feature_pyramid_network import FeaturePyramidNetwork
import torchvision.transforms as transforms

from dataset import FeatureDataset
import networks

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# GP (Approximation)

In [2]:
def check_group(group: str, classification: list):
    if group == "Old&Asian":
        return classification[2] == "Asian" and classification[4] == "70+"
    elif group == "Young&Asian":
        return classification[2] == "Asian" and classification[4] == "20-29"
    elif group == "Old&non-Asian":
        return classification[2] != "Asian" and classification[4] == "70+"
    elif group == "Young&non-Asian":
        return classification[2] != "Asian" and classification[4] == "20-29"
    return False

In [102]:
csv_file = "results/half+repetition/test_outputs.csv"

correct_predictions = [0,0,0,0]
total_predictions = [0,0,0,0]
with open(csv_file, 'r') as f:
    csv_reader = csv.reader(f)
    for row in csv_reader:
        if re.search('Old&Asian', row[0]):
            if check_group("Old&Asian", row):
                correct_predictions[0] += 1
            total_predictions[0] += 1
        elif re.search('Old&non-Asian', row[0]):
            if check_group("Old&non-Asian", row):
                correct_predictions[1] += 1
            total_predictions[1] += 1
        elif re.search('Young&Asian', row[0]):
            if check_group("Young&Asian", row):
                correct_predictions[2] += 1
            total_predictions[2] += 1
        elif re.search('Young&non-Asian', row[0]):
            if check_group("Young&non-Asian", row):
                correct_predictions[3] += 1
            total_predictions[3] += 1

print(correct_predictions[0]/total_predictions[0])
print(correct_predictions[1]/total_predictions[1])
print(correct_predictions[2]/total_predictions[2])
print(correct_predictions[3]/total_predictions[3])

0.825
0.6730769230769231
0.6788461538461539
0.8730769230769231


# Torch Fidelity (Output Images)

In [56]:
model_fair_4 = torchvision.models.resnet34(pretrained=True)
model_fair_4.fc = nn.Linear(model_fair_4.fc.in_features, 18)
model_fair_4.load_state_dict(torch.load('FairFace/fair_face_models/res34_fair_align_multi_4_20190809.pt'))
return_nodes = {"avgpool": "avgpool"}
fair_4_features = create_feature_extractor(model_fair_4, return_nodes=return_nodes)

C:\Users\colin\AppData\Local\Temp\ipykernel_7252\1591604211.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_fair_4.load_state_dict(torch.load('FairFace/fair_face_m

In [89]:
true_paths = []
for dir in Path('test').glob('**/*'):
    true_paths.extend(dir.glob('*.png'))

out_paths = []
for dir in Path('results/all').glob('**/*'):
    if dir.is_dir():
        out_paths.extend(dir.glob('*.png'))

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

true_features = FeatureDataset(true_paths, 224, fair_4_features, 'cuda', transform)
out_features = FeatureDataset(out_paths, 224, fair_4_features, 'cuda', transform)

In [114]:
metrics = torch_fidelity.calculate_metrics(
    input1='test/Young&Asian',
    input2='results/all/Young&Asian',
    cuda=True,
    fid=True,
    kid=True,
    kid_subset_size=520,
    prc=True,
    verbose=True
)

print(metrics)

Creating feature extractor "inception-v3-compat" with features ['2048']
Extracting features from input1
Looking for samples non-recursivelty in "test/Young&Asian" with extensions png,jpg,jpeg
Found 520 samples
Processing samples                                                        
Extracting features from input2
Looking for samples non-recursivelty in "results/all/Young&Asian" with extensions png,jpg,jpeg
Found 520 samples
Processing samples                                                        
Frechet Inception Distance: 114.53709028044153
                                                                                

{'frechet_inception_distance': 114.53709028044153, 'kernel_inception_distance_mean': 0.09608116555630854, 'kernel_inception_distance_std': 1.6334298521730888e-07}


Kernel Inception Distance: 0.09608116555630854 ± 1.6334298521730888e-07
